In [1]:
# Author: Vanshika Gupta
# Jointly with: Prof Chrysafis Vogiatzis

In [2]:
from gurobipy import *
import numpy as np
import matplotlib.pyplot as plt

#### Data

In [3]:
# number of Flights
nFlights = 6

# Reservations/Bookings
reserve = [110, 130, 103, 140, 135, 0]

# airline capacity #last kept like bigM as other airlines can be used indefinitely as per the question
capacity = [100, 100, 100, 150, 150, 2000]

# fixed and variable Penality
fPenalty = 200
vPenalty = 80

# initial configuration of passengers y-> initial scheduled flight x-> final sheduled flight
initialFlight = np.diag(np.array(reserve))
print(initialFlight)

[[110   0   0   0   0   0]
 [  0 130   0   0   0   0]
 [  0   0 103   0   0   0]
 [  0   0   0 140   0   0]
 [  0   0   0   0 135   0]
 [  0   0   0   0   0   0]]


#### Variables

In [4]:
# final configuration of passengers y-> initial scheduled flight x-> final sheduled flight
finalFlight = {}

#### Create Model & Variables

In [5]:
model=Model("facilities")

finalFlight = model.addVars(nFlights, nFlights, vtype = GRB.INTEGER)

Academic license - for non-commercial use only - expires 2022-09-08
Using license file /home/vanshika/gurobi.lic


#### Constraint

In [6]:
# number of passengers should not exceed capacity
for i in range(nFlights):
    expr = 0
    for j in range(nFlights):
        expr += finalFlight[j,i]
    model.addConstr(expr <= capacity[i])
    
# number of passenger originally in a flight should equal the reservations made i.e.row sum=reservations 
for i in range(nFlights):
    expr = 0
    for j in range(nFlights):
        expr += finalFlight[i,j]
    model.addConstr(expr == reserve[i])
    
# lower triangle should be 0 (since all solutions are +ve by default, we can also say sum of lower triangle is 0)    
for i in range(nFlights):
    expr = 0
    for j in range(i):
        expr += finalFlight[i,j]
    model.addConstr(expr == 0)

#### Objective

In [7]:
obj = 0
for i in range(nFlights):
    for j in range(i+1,nFlights):
        obj += (fPenalty + vPenalty*(j-i-1))*finalFlight[i,j]

model.setObjective(obj)

#### Optimize

In [8]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 18 rows, 36 columns and 87 nonzeros
Model fingerprint: 0x33cbc7a8
Variable types: 0 continuous, 36 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Found heuristic solution: objective 170240.00000
Presolve removed 10 rows and 21 columns
Presolve time: 0.00s
Presolved: 8 rows, 15 columns, 28 nonzeros
Variable types: 0 continuous, 15 integer (0 binary)

Root relaxation: objective 1.668000e+04, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    16680.000000 16680.0000  0.00%     -    0s

Explored 0 nodes (4 simplex iterations) in 0.01 seconds
Thread 

#### Print FInal Solution

In [9]:
print("Final Flight Configuration \n")
for i in range(nFlights):
    print('\t', i, ')', end='')
print('\n')
for i in range(nFlights):
    print(i, ')\t', end='')
    for j in range(nFlights):
        print(round(finalFlight[i,j].X),'\t', end='')
    print('\n')

Final Flight Configuration 

	 0 )	 1 )	 2 )	 3 )	 4 )	 5 )

0 )	100 	0 	0 	0 	10 	0 	

1 )	0 	100 	0 	10 	5 	15 	

2 )	0 	0 	100 	0 	0 	3 	

3 )	0 	0 	0 	140 	0 	0 	

4 )	0 	0 	0 	0 	135 	0 	

5 )	0 	0 	0 	0 	0 	0 	



In [10]:
print("Min Cost incurred to the airlines: $", model.getObjective().getValue())

Min Cost incurred to the airlines: $ 16680.0
